In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image


# Set reproducibility
np.random.seed(42)
tf.random.set_seed(42)


def crop_perfusion_image(input_path, crop_box=(360, 19, 957, 920)):
    img = Image.open(input_path)
    cropped_img = img.crop(crop_box)
    return cropped_img


def load_and_preprocess_data(base_path):
    data, labels, dropped_patients = [], [], []
    target_images = ['AC QPS (clear)', 'NAC QPS (clear)', 'Stress QGS (clear)', 'Rest QGS (clear)']
    crop_box = (360, 19, 957, 920)

    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if not os.path.isdir(folder_path):
            continue

        if '(' in folder and ')' in folder:
            label = folder.split('(')[-1].split(')')[0].strip()
            patient_id = folder.split('(')[0].strip()
        else:
            continue

        patient_images, all_found = [], True
        for image_type in target_images:
            found = False
            for file in os.listdir(folder_path):
                if image_type in file and 'segment' not in file.lower():
                    try:
                        img = Image.open(os.path.join(folder_path, file))
                        img = img.crop(crop_box).resize((224, 224))
                        img_array = np.array(img) / 255.0
                        patient_images.append(img_array)
                        found = True
                        break
                    except Exception:
                        continue
            if not found:
                all_found = False
                break

        if all_found:
            data.extend(patient_images)
            labels.extend([label] * len(patient_images))
        else:
            dropped_patients.append(patient_id)

    print(f"\nDropped patients: {dropped_patients}")
    print(f"Total images loaded: {len(data)}")
    return np.array(data), np.array(labels)


def perform_augmentation_oversampling(X, y, le, target_count=None):
    unique_classes, class_counts = np.unique(y, return_counts=True)
    class_distribution = dict(zip(unique_classes, class_counts))

    print("Original class distribution:")
    for class_idx, count in class_distribution.items():
        print(f"Class {le.classes_[class_idx]}: {count} samples")

    if target_count is None:
        target_count = max(class_counts)
        print(f"\nTarget count per class: {target_count} (maximum class count)")
    else:
        print(f"\nTarget count per class: {target_count} (user-specified)")

    augmentation = tf.keras.Sequential([
        layers.RandomRotation(0.2),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.2),
        layers.GaussianNoise(0.02)
    ])

    X_balanced = list(X)
    y_balanced = list(y)

    for class_idx in unique_classes:
        class_indices = np.where(y == class_idx)[0]
        X_class = X[class_indices]
        y_class = y[class_indices]
        samples_needed = target_count - len(X_class)

        if samples_needed > 0:
            print(f"Augmenting class {le.classes_[class_idx]}: adding {samples_needed} samples")

            batch_size = min(32, samples_needed)
            while samples_needed > 0:
                current_batch_size = min(batch_size, samples_needed)
                selected_indices = np.random.choice(len(X_class), size=current_batch_size, replace=True)
                images_to_augment = tf.convert_to_tensor(X_class[selected_indices], dtype=tf.float32)
                augmented_batch = augmentation(images_to_augment).numpy()
                augmented_batch = np.clip(augmented_batch, 0, 1)

                X_balanced.extend(augmented_batch)
                y_balanced.extend([class_idx] * current_batch_size)
                samples_needed -= current_batch_size

    X_balanced = np.array(X_balanced)
    y_balanced = np.array(y_balanced)

    unique_classes_new, class_counts_new = np.unique(y_balanced, return_counts=True)
    print("\nClass distribution after augmentation:")
    for class_idx, count in zip(unique_classes_new, class_counts_new):
        print(f"Class {le.classes_[class_idx]}: {count} samples")

    return X_balanced, y_balanced


def create_model(num_classes, num_conv_layers):
    local_weights_path = '/kaggle/input/weight/resnet50v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
    base_model = ResNet50V2(
        include_top=False,
        weights=local_weights_path,
        input_shape=(224, 224, 3)
)    
    base_model.trainable = False

    model = models.Sequential([base_model])
    for i in range(num_conv_layers):
        model.add(layers.Conv2D(32 * (2 ** i), (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D((2, 2), padding='same'))
        model.add(layers.Dropout(0.25))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l1_l2(1e-4, 1e-4)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l1_l2(1e-4, 1e-4)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model


def plot_confusion_matrix(cm, classes, title, filename):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()


def plot_training_history(history, title_prefix, filename_prefix):
    # Plot Accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{title_prefix} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{filename_prefix}_accuracy.png')
    plt.close()

    # Plot Loss
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{title_prefix} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{filename_prefix}_loss.png')
    plt.close()


def main():
    base_path = '/kaggle/input/mpi-images/MPI-Images'
    X, y = load_and_preprocess_data(base_path)

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    num_classes = len(le.classes_)

    X_balanced, y_balanced = perform_augmentation_oversampling(X, y_encoded, le)

    X_train, X_test, y_train, y_test = train_test_split(
        X_balanced, y_balanced, test_size=0.2, stratify=y_balanced, random_state=42
    )

    batch_sizes = [8, 16, 32]
    learning_rates = [1e-3, 1e-4]
    conv_layers_list = [1, 2, 3, 4]

    results = []
    best_val_acc = 0

    for num_conv in conv_layers_list:
        for batch_size in batch_sizes:
            for lr in learning_rates:
                print(f"\n--- Training: {num_conv} Conv2D, Batch Size: {batch_size}, LR: {lr} ---")
                tf.keras.backend.clear_session()

                model = create_model(num_classes, num_conv)
                model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                              loss='sparse_categorical_crossentropy',
                              metrics=['accuracy'])

                callbacks = [
                    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                    ModelCheckpoint(f"best_ResNet50V2_{num_conv}Conv_B{batch_size}_LR{lr}.h5",
                                    monitor='val_accuracy', save_best_only=True)
                ]

                history = model.fit(
                    ImageDataGenerator().flow(X_train, y_train, batch_size=batch_size),
                    validation_data=(X_test, y_test),
                    epochs=20,
                    callbacks=callbacks,
                    verbose=1
                )

                val_acc = max(history.history['val_accuracy'])
                print(f"Best Val Accuracy: {val_acc:.4f}")

                # ➡ Plot accuracy and loss
                plot_training_history(history,
                                      title_prefix=f'{num_conv}Conv B{batch_size} LR{lr}',
                                      filename_prefix=f'{num_conv}Conv_B{batch_size}_LR{lr}')

                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    model.save("best_overall_model.h5")

                y_pred = np.argmax(model.predict(X_test), axis=1)
                report = classification_report(y_test, y_pred, target_names=le.classes_, output_dict=True)
                cm = confusion_matrix(y_test, y_pred)
                plot_confusion_matrix(cm, le.classes_,
                                      f'Confusion Matrix {num_conv}Conv B{batch_size} LR{lr}',
                                      f'CM_{num_conv}Conv_B{batch_size}_LR{lr}.png')

                results.append({
                    'Conv2D_Layers': num_conv,
                    'Batch_Size': batch_size,
                    'Learning_Rate': lr,
                    'Best_Val_Accuracy': val_acc
                })

    df_results = pd.DataFrame(results)
    df_results.to_csv("grid_search_results.csv", index=False)
    print("\n=== Grid Search Completed ===")
    print(df_results.sort_values(by='Best_Val_Accuracy', ascending=False))


if __name__ == "__main__":
    main()


2025-05-16 09:16:00.284386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747386960.463806      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747386960.516604      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



Dropped patients: ['197184', '249172', '117367', '140942']
Total images loaded: 336
Original class distribution:
Class Infarction: 16 samples
Class Ischemia: 72 samples
Class Normal: 248 samples

Target count per class: 248 (maximum class count)


I0000 00:00:1747386988.619412      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747386988.620113      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Augmenting class Infarction: adding 232 samples
Augmenting class Ischemia: adding 176 samples

Class distribution after augmentation:
Class Infarction: 248 samples
Class Ischemia: 248 samples
Class Normal: 248 samples

--- Training: 1 Conv2D, Batch Size: 8, LR: 0.001 ---
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1747387006.073400      97 service.cc:148] XLA service 0x246c2db0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747387006.074338      97 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747387006.074351      97 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747387007.559982      97 cuda_dnn.cc:529] Loaded cuDNN version 90300


...

38/38 ━━━━━━━━━━━━━━━━━━━━ 36s 444ms/step - accuracy: 0.3427 - loss: 2.1994 - val_accuracy: 0.4631 - val_loss: 1.8491
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.4265 - loss: 1.9261 - val_accuracy: 0.6376 - val_loss: 1.8009
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - accuracy: 0.4798 - loss: 1.8848 - val_accuracy: 0.6577 - val_loss: 1.7166
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - accuracy: 0.5157 - loss: 1.7672 - val_accuracy: 0.6644 - val_loss: 1.6150
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 0.5528 - loss: 1.7318 - val_accuracy: 0.5369 - val_loss: 1.6185
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.6304 - loss: 1.5330 - val_accuracy: 0.6644 - val_loss: 1.4805
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.6794 - loss: 1.5052 - val_accuracy: 0.7517 - val_loss: 1.3406
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.6589 - loss: 1.5149 - val_accuracy: 0.7584 - val_loss: 

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 956ms/step - accuracy: 0.4605 - loss: 2.0393 - val_accuracy: 0.5302 - val_loss: 1.8162
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - accuracy: 0.6957 - loss: 1.3901 - val_accuracy: 0.5772 - val_loss: 1.6569
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - accuracy: 0.8124 - loss: 1.2718 - val_accuracy: 0.6443 - val_loss: 1.3550
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.8249 - loss: 1.2394 - val_accuracy: 0.6711 - val_loss: 1.3230
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.8465 - loss: 1.1005 - val_accuracy: 0.7718 - val_loss: 1.2618
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - accuracy: 0.8967 - loss: 1.0275 - val_accuracy: 0.8456 - val_loss: 1.1406
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - accuracy: 0.8808 - loss: 1.0197 - val_accuracy: 0.8054 - val_loss: 1.1994
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - accuracy: 0.9068 - loss: 0.9963 - val_accuracy: 0.8725 - val

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 971ms/step - accuracy: 0.4238 - loss: 2.0213 - val_accuracy: 0.4832 - val_loss: 1.8247
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - accuracy: 0.4078 - loss: 1.9312 - val_accuracy: 0.6174 - val_loss: 1.7843
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.5321 - loss: 1.7813 - val_accuracy: 0.6443 - val_loss: 1.7119
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - accuracy: 0.5549 - loss: 1.7236 - val_accuracy: 0.6644 - val_loss: 1.6362
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 154ms/step - accuracy: 0.6120 - loss: 1.5710 - val_accuracy: 0.6711 - val_loss: 1.5712
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - accuracy: 0.6661 - loss: 1.4788 - val_accuracy: 0.6913 - val_loss: 1.5112
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.6208 - loss: 1.5361 - val_accuracy: 0.7248 - val_loss: 1.4530
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 142ms/step - accuracy: 0.6506 - loss: 1.4790 - val_accuracy: 0.7651 - val